# Sentinel-2 Harmonized Image Download from Google Earth Engine

## 1. Setting the scene  

### 1.1. Import required libraries

In [1]:
import ee
import geemap
import geopandas as gpd

### 1.2.Define paths to datasets

In [2]:
m_root=r"C:/Users/VICTUS/Documents/MG(2)/programacion_sig/notebook/Workshop/PS/data"
aoi=m_root+r"/jerico.gpkg" # Path to the vector file that contains the AOI

### 1.3. Authenticate and Initialise Earth Engine

In [3]:
# Attempt to initialise the Earth Engine API
try:
    ee.Initialize()

    # If initialisation fails (likely because the user is not yet authenticated), prompt the user to log in using their Google account
except Exception as e:
    # If Earth Engine is not yet authenticated, prompt the user to sign in via a browser
    # This step ensures proper authorisation using a Google account
    ee.Authenticate()
    
    # Re-initialise Earth Engine after successful authentication
    ee.Initialize()

### 1.4. Read the Area of Interest   

Load a vector file containing the Area of Interest (AOI) using `geopandas`, and then converts the geometry from the Geopackage into an Earth Engine geometry object. This is necessary because Earth Engine requires its own geometry format for spatial operations.

In [4]:
# Read shapefile with geopandas
gdf_aoi = gpd.read_file(aoi)
gdf_aoi = gdf_aoi.to_crs("EPSG:4326")  # Ensure the CRS is WGS84
# Convert to Earth Engine geometry
aoi_geom = gdf_aoi.geometry[0].__geo_interface__
aoi_ee = ee.Geometry(aoi_geom)

## 2. Filter Sentinel-2 Harmonized Imagery

We will now search for Sentinel-2 Surface Reflectance (Level 2) imagery using the COPERNICUS/S2_SR_HARMONIZED collection:  

- Filter by date   
- Filter by cloud coverage  
- Filter by location (AOI) You can read more about this collection and explore its metadata, bands, and usage guidance here:
COPERNICUS/S2_SR_HARMONIZED – Earth Engine Data Catalog

In [5]:
# Define the temporal range for image selection
start_date = '2022-01-01'  # Start of the desired date range
end_date = '2022-12-31'    # End of the desired date range

# Load the Sentinel-2 Harmonised Surface Reflectance image collection
# and apply filters to narrow down the search
collection = (
    ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
    .filterDate(start_date, end_date)              # Filter images by date
    .filterBounds(aoi_ee)                          # Filter images by location (our AOI)
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))  # Keep only images with <10% cloud cover
)

# Generate a median composite image from the filtered collection
# This reduces noise and cloud effects by taking the median value per pixel
composite = collection.median().clip(aoi_ee)  # Clip to the AOI to minimise processing size

## 3. Visualise the Image composite

Before exporting the processed image, it’s good practice to preview the result and ensure it covers the expected area, with adequate quality.  

Use geemap to create an interactive map and overlay the Sentinel-2 RGB composite (bands B4, B3, B2).

In [6]:
# Create an interactive map centred on the AOI (Area of Interest)

# We use geemap.Map(), which is based on ipyleaflet, to build an interactive map
# The 'center' parameter takes [latitude, longitude] as input

# To calculate the centre of the AOI:
# - gdf_aoi.geometry[0] accesses the first (and in our case, only) geometry in the GeoDataFrame
# - .centroid returns the centre point (as a shapely Point object) of the polygon
# - .y and .x extract the latitude and longitude coordinates of the centroid respectively

Map = geemap.Map(
    center=[gdf_aoi.geometry[0].centroid.y, gdf_aoi.geometry[0].centroid.x],  # [lat, lon]
    zoom=13  # Set initial zoom level (adjust as needed)
)
# Add the composite to the map with RGB bands (B4 = red, B3 = green, B2 = blue)

rgb_vis_params = {
    'min': 100,    # Slight offset to reduce haze
    'max': 2000,   # Higher max to improve contrast
    'bands': ['B4', 'B3', 'B2']  # RGB bands
}

# Add the Sentinel-2 RGB composite to the map
# - 'composite' is the image to be displayed
# - 'rgb_vis_params' defines which bands to use (B4 = red, B3 = green, B2 = blue) and sets the brightness/contrast range using 'min' and 'max'
# - The third argument is the label that appears in the map layer control
Map.addLayer(composite, rgb_vis_params, "Sentinel-2 RGB Composite (Brightened)")

# Add the Area of Interest (AOI) boundary to the map as a vector overlay
Map.addLayer(aoi_ee, {}, "AOI Boundary")

# Enable map layer controls (checkboxes and layer ordering)
Map.addLayerControl()

# Display the map
Map

Map(center=[5.769820536039182, -75.7648825557432], controls=(WidgetControl(options=['position', 'transparent_b…

### 3.1. Explore the names of all bands in the composite image

The Sentinel-2 composite image contains multiple spectral bands, each capturing reflectance in a different portion of the electromagnetic spectrum. Before creating custom visualisations, it's important to know which spectral bands are present in the image as well as the min and max values of each of them.  

The followng lines:  

- List the available band names in the composite.  
- Calculate the minimum and maximum reflectance values** for each band over the area of interest (AOI).  

These statistics are useful when setting min and max values for visualisation. Because each band can have a different reflectance range, using the same visualisation parameters across all bands might lead to overly dark or saturated images.

In [7]:
# Print the names of all bands available in the composite image
band_names = composite.bandNames().getInfo()
print("Available bands in the Sentinel-2 composite:")
print(band_names)

# Display min and max values for each band
print("\n Min and Max values per band:")

# Loop through each band and print its value range (with safety check)
for band in band_names:
    band_img = composite.select(band)
    stats = band_img.reduceRegion(
        reducer=ee.Reducer.minMax(),
        geometry=aoi_ee,
        scale=10,
        maxPixels=1e9
    ).getInfo()
    
    min_val = stats.get(f"{band}_min")
    max_val = stats.get(f"{band}_max")
    
    if min_val is not None and max_val is not None:
        print(f"  {band}: min = {min_val:.2f}, max = {max_val:.2f}")
    else:
        print(f"  {band}: No data available in AOI")

Available bands in the Sentinel-2 composite:
['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12', 'AOT', 'WVP', 'SCL', 'TCI_R', 'TCI_G', 'TCI_B', 'MSK_CLDPRB', 'MSK_SNWPRB', 'QA10', 'QA20', 'QA60', 'MSK_CLASSI_OPAQUE', 'MSK_CLASSI_CIRRUS', 'MSK_CLASSI_SNOW_ICE']

 Min and Max values per band:
  B1: min = 30.00, max = 8624.00
  B2: min = 0.00, max = 11328.00
  B3: min = 0.00, max = 9064.00
  B4: min = 2.00, max = 9544.00
  B5: min = 51.00, max = 9524.00
  B6: min = 20.00, max = 9545.00
  B7: min = 79.00, max = 9488.00
  B8: min = 106.00, max = 10448.00
  B8A: min = 97.00, max = 9426.00
  B9: min = 270.00, max = 15003.00
  B11: min = 137.00, max = 8441.00
  B12: min = 70.00, max = 7329.00
  AOT: min = 75.00, max = 216.00
  WVP: min = 139.00, max = 3981.00
  SCL: min = 2.00, max = 10.00
  TCI_R: min = 1.00, max = 255.00
  TCI_G: min = 3.00, max = 255.00
  TCI_B: min = 1.00, max = 255.00
  MSK_CLDPRB: min = 0.00, max = 100.00
  MSK_SNWPRB: min = 0.00, max = 0.00
  QA

### 3.2. Stretching Bands Individually for Enhanced Visualisation  

So far, we have visualised our Sentinel-2 image using a single brightness/contrast stretch (min/max) applied uniformly to all bands.  

However, each spectral band may have a different reflectance range. Applying a single stretch to all bands may result in an overly dark or overly bright image.  

To enhance visual contrast, we can apply a custom stretch to each band individually. This involves:  

- Selecting each band separately  
- Applying a normalisation (e.g. from 0–3000 to 0–1) using .unitScale()    
- Merging the bands back together for display.  

Let’s apply custom visual scaling to the RGB bands (B4, B3, B2).

In [8]:
Map2 = geemap.Map(
    center=[gdf_aoi.geometry[0].centroid.y, gdf_aoi.geometry[0].centroid.x],  # [lat, lon]
    zoom=13  # Set initial zoom level (adjust as needed)
)
# Define custom min/max values for each RGB band
# These values can be adjusted based on the actual reflectance range of each band
stretch_params = {
    'B4': {'min': 0, 'max': 3000},  # Red band
    'B3': {'min': 0, 'max': 2500},  # Green band
    'B2': {'min': 0, 'max': 2000}   # Blue band
}

# Apply per-band linear stretch using .unitScale(min, max), which rescales values to 0–1
b4_scaled = composite.select('B4').unitScale(stretch_params['B4']['min'], stretch_params['B4']['max'])
b3_scaled = composite.select('B3').unitScale(stretch_params['B3']['min'], stretch_params['B3']['max'])
b2_scaled = composite.select('B2').unitScale(stretch_params['B2']['min'], stretch_params['B2']['max'])

# Combine the stretched bands into a new composite image
rgb_stretched = ee.Image.cat([b4_scaled, b3_scaled, b2_scaled])

# Add the new composite to the interactive map
Map2.addLayer(rgb_stretched, {'min': 0, 'max': 1, 'bands': ['B4', 'B3', 'B2']}, 'Stretched RGB (per band)')

# Display the updated map
Map2

Map(center=[5.769820536039182, -75.7648825557432], controls=(WidgetControl(options=['position', 'transparent_b…

### ✏️ Now it's your turn  

You’ve just learnt how to enhance visualisation by applying a custom stretch per band. Now, let’s create a false colour composite using the following Sentinel-2 bands:  

- B8 (Near Infrared)  
- B4 (Red)  
- B3 (Green)   

False colour composites are useful for vegetation monitoring, as healthy vegetation reflects strongly in the NIR.

In [9]:
Map2 = geemap.Map(
    center=[gdf_aoi.geometry[0].centroid.y, gdf_aoi.geometry[0].centroid.x],  # [lat, lon]
    zoom=13  # Set initial zoom level (adjust as needed)
)
# Define custom min/max values for each RGB band
# These values can be adjusted based on the actual reflectance range of each band
stretch_params = {
    'B8': {'min': 0, 'max': 3000},  # Near infrared band
    'B4': {'min': 0, 'max': 3000},  # Red band
    'B3': {'min': 0, 'max': 2500}   # Green band
}

# Apply per-band linear stretch using .unitScale(min, max), which rescales values to 0–1
b4_scaled = composite.select('B8').unitScale(stretch_params['B8']['min'], stretch_params['B8']['max'])
b3_scaled = composite.select('B4').unitScale(stretch_params['B4']['min'], stretch_params['B4']['max'])
b2_scaled = composite.select('B3').unitScale(stretch_params['B3']['min'], stretch_params['B3']['max'])

# Combine the stretched bands into a new composite image
rgb_stretched = ee.Image.cat([b4_scaled, b3_scaled, b2_scaled])

# Add the new composite to the interactive map
Map2.addLayer(rgb_stretched, {'min': 0, 'max': 1, 'bands': ['B8', 'B4', 'B3']}, 'Stretched False colour (per band)')

# Display the updated map
Map2

Map(center=[5.769820536039182, -75.7648825557432], controls=(WidgetControl(options=['position', 'transparent_b…

## 4. Exporting the Composite Image to Google Drive  

Once the image has been processed and reviewed, we can export it to Google Drive, which is suitable for saving large images.  

Exporting to Google Drive is recommended for high-resolution or large-area composites, as it avoids memory limitations commonly encountered when downloading to the local environment.  

The following code sends the composite image (clipped to the AOI) as a GeoTIFF file to your Google Drive. You will find it in a folder called earthengine by default, unless otherwise specified.

In [10]:
# Export the composite image to Google Drive
task = ee.batch.Export.image.toDrive(
    image=composite,
    description='Sentinel2_Composite_Export',
    folder='earthengine',            # Destination folder in Google Drive
    fileNamePrefix='sentinel2_rgb', # File name without extension
    region=aoi_ee,                   # Area to export
    scale=10,                        # Set a very high limit to allow exporting large images
                                     # Earth Engine requires this to avoid exporting by mistake very large rasters
                                     # The default limit is 1e8 (100 million pixels), so we override it here

    maxPixels=1e13,                  # Allow large exports
    fileFormat='GeoTIFF'
)

# Start the export task
task.start()

print("Export task started. Monitor progress in the Earth Engine Tasks tab.")

Export task started. Monitor progress in the Earth Engine Tasks tab.


### ✏️ Now it's your turn: Work with a Landsat Collection and Download a False Colour Image  

Let’s now apply what you've learned to a new satellite image collection: Landsat.  

1. Search and choose a Landsat image collection available in Earth Engine.  

2. Inspect the available bands in the collection and identify:  

- Near Infrared (NIR)  
- Red  
- Green  

3. Load a filtered image collection** using:

- A relevant date range  
- Your existing Area of Interest  
- An appropriate cloud cover filter  

4. Generate a composite using a different function to median().  

5. Visualise the image on the map using a false colour RGB combination (e.g. NIR-Red-Green).  

6. Export the image to your computer (not Google Drive) in GeoTIFF format using geemap.ee_export_image() as you did with the SRTM.  

Give it a try and see how Landsat compares to Sentinel-2!